# 🌱 CO2 배출량 데이터 시각화
## 환경 친화적 차량 분석 (간단 버전)
작성일: 2025년 8월 19일

In [1]:
# 라이브러리 임포트
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

print("📦 라이브러리 로드 완료!")

📦 라이브러리 로드 완료!


In [2]:
# 데이터 로드
co2_data = pd.read_csv("dataset/CO2_Emissions.csv")

print("🌱 CO2 배출량 데이터 정보")
print(f"총 차량 수: {len(co2_data):,}대")
print(f"제조사 수: {co2_data['Make'].nunique()}개")
print(f"평균 CO2 배출량: {co2_data['CO2 Emissions(g/km)'].mean():.1f} g/km")

co2_data.head()

🌱 CO2 배출량 데이터 정보
총 차량 수: 7,385대
제조사 수: 42개
평균 CO2 배출량: 250.6 g/km


,Make,Model,Vehicle Class,Engine Size(L),Cylinders,Transmission,Fuel Type,Fuel Consumption City (L/100 km),Fuel Consumption Hwy (L/100 km),Fuel Consumption Comb (L/100 km),Fuel Consumption Comb (mpg),CO2 Emissions(g/km)
0,ACURA,ILX,COMPACT,2.0,4,AS5,Z,9.9,6.7,8.5,33,196
1,ACURA,ILX,COMPACT,2.4,4,M6,Z,11.2,7.7,9.6,29,221
2,ACURA,ILX HYBRID,COMPACT,1.5,4,AV7,Z,6.0,5.8,5.9,48,136
3,ACURA,MDX 4WD,SUV - SMALL,3.5,6,AS6,Z,12.7,9.1,11.1,25,255
4,ACURA,RDX AWD,SUV - SMALL,3.5,6,AS6,Z,12.1,8.7,10.6,27,244


## 🌟 시각화 1: 버블 차트 - 엔진 크기 vs CO2 배출량

In [3]:
# 상위 10개 제조사만 선택
top_makes = co2_data['Make'].value_counts().head(10).index
filtered_data = co2_data[co2_data['Make'].isin(top_makes)]

# 버블 차트 생성
fig1 = px.scatter(
    filtered_data,
    x='Engine Size(L)',
    y='CO2 Emissions(g/km)',
    size='Fuel Consumption Comb (mpg)',
    color='Make',
    title='🌍 엔진 크기 vs CO2 배출량 (버블: 연비)',
    hover_data=['Model', 'Vehicle Class'],
    size_max=15
)

# 친환경 기준선 추가
fig1.add_hline(y=200, line_dash="dash", line_color="red", 
              annotation_text="친환경 기준 (200g/km)")

fig1.update_layout(width=800, height=500)
fig1.show()

## 🏆 시각화 2: 제조사별 환경 성능 랭킹

In [4]:
# 제조사별 평균 CO2 계산 (5대 이상)
make_stats = co2_data.groupby('Make').agg({
    'CO2 Emissions(g/km)': ['mean', 'count']
}).round(1)

make_stats.columns = ['평균_CO2', '차량수']
make_stats = make_stats[make_stats['차량수'] >= 5]
top_15 = make_stats.sort_values('평균_CO2').head(15)

# 가로 막대 차트
fig2 = px.bar(
    x=top_15['평균_CO2'],
    y=top_15.index,
    orientation='h',
    title='🏆 친환경 제조사 TOP 15 (낮은 CO2 순)',
    labels={'x': 'CO2 배출량 (g/km)', 'y': '제조사'},
    color=top_15['평균_CO2'],
    color_continuous_scale='RdYlGn_r'
)

fig2.add_vline(x=200, line_dash="dash", line_color="red")
fig2.update_layout(width=800, height=600)
fig2.show()

print(f"🏆 가장 친환경 제조사: {top_15.index[0]} ({top_15.iloc[0]['평균_CO2']:.1f} g/km)")

🏆 가장 친환경 제조사: SMART (151.4 g/km)


## 📊 시각화 3: 차량 클래스별 CO2 분포

In [5]:
# 박스플롯
fig3 = px.box(
    co2_data,
    x='Vehicle Class',
    y='CO2 Emissions(g/km)',
    title='📊 차량 클래스별 CO2 배출량 분포',
    color='Vehicle Class'
)

fig3.add_hline(y=200, line_dash="dash", line_color="red")
fig3.update_xaxes(tickangle=45)
fig3.update_layout(showlegend=False, width=900, height=500)
fig3.show()

## 🌈 시각화 4: 연비 vs CO2 상관관계

In [6]:
# 상관계수 계산
correlation = co2_data['Fuel Consumption Comb (mpg)'].corr(
    co2_data['CO2 Emissions(g/km)']
)

print(f"📈 연비와 CO2 상관계수: {correlation:.3f}")

# 산점도
fig4 = px.scatter(
    co2_data,
    x='Fuel Consumption Comb (mpg)',
    y='CO2 Emissions(g/km)',
    color='Vehicle Class',
    title=f'⛽ 연비 vs CO2 배출량 (상관계수: {correlation:.3f})',
    opacity=0.6
)

fig4.update_layout(width=800, height=500)
fig4.show()

📈 연비와 CO2 상관계수: -0.907


## 🌿 시각화 5: 친환경 차량 분석

In [7]:
# 하이브리드 차량 식별
co2_data['Is_Hybrid'] = co2_data['Model'].str.contains('HYBRID', case=False, na=False)

# 친환경 차량 (CO2 < 200g/km)
eco_friendly = co2_data[co2_data['CO2 Emissions(g/km)'] < 200]

print(f"🌱 친환경 차량 수: {len(eco_friendly)}대 ({len(eco_friendly)/len(co2_data)*100:.1f}%)")
print(f"🔋 하이브리드 차량 수: {co2_data['Is_Hybrid'].sum()}대")

# 하이브리드 vs 일반 차량 비교
hybrid_avg = co2_data[co2_data['Is_Hybrid']]['CO2 Emissions(g/km)'].mean()
conventional_avg = co2_data[~co2_data['Is_Hybrid']]['CO2 Emissions(g/km)'].mean()

comparison_data = pd.DataFrame({
    'Type': ['Hybrid', 'Conventional'],
    'Avg_CO2': [hybrid_avg, conventional_avg]
})

fig5 = px.bar(
    comparison_data,
    x='Type',
    y='Avg_CO2',
    title='🔋 하이브리드 vs 일반 차량 CO2 비교',
    color='Avg_CO2',
    color_continuous_scale='RdYlGn_r',
    text='Avg_CO2'
)

fig5.update_traces(texttemplate='%{text:.1f}g/km', textposition='outside')
fig5.update_layout(width=600, height=400)
fig5.show()

print(f"💚 하이브리드 효과: {conventional_avg - hybrid_avg:.1f} g/km 절약")

🌱 친환경 차량 수: 1493대 (20.2%)
🔋 하이브리드 차량 수: 135대


💚 하이브리드 효과: 93.8 g/km 절약


## 🏆 친환경 차량 추천 TOP 10

In [8]:
# 친환경 차량 TOP 10
top_eco = eco_friendly.nsmallest(10, 'CO2 Emissions(g/km)')[
    ['Make', 'Model', 'Vehicle Class', 'Engine Size(L)', 
     'CO2 Emissions(g/km)', 'Fuel Consumption Comb (mpg)', 'Is_Hybrid']
].round(1)

print("\n🌟 친환경 차량 추천 TOP 10:")
print("=" * 80)
for i, row in top_eco.iterrows():
    hybrid_mark = "🔋" if row['Is_Hybrid'] else "🚗"
    print(f"{hybrid_mark} {row['Make']} {row['Model']} - {row['CO2 Emissions(g/km)']}g/km - {row['Fuel Consumption Comb (mpg)']}mpg")

top_eco


🌟 친환경 차량 추천 TOP 10:
🚗 HYUNDAI IONIQ BLUE - 96g/km - 69mpg
🚗 HYUNDAI IONIQ BLUE - 96g/km - 69mpg
🚗 HYUNDAI IONIQ Blue - 96g/km - 69mpg
🚗 HYUNDAI IONIQ Blue - 96g/km - 69mpg
🚗 HYUNDAI IONIQ - 99g/km - 67mpg
🚗 HYUNDAI IONIQ - 102g/km - 66mpg
🚗 HYUNDAI IONIQ - 103g/km - 64mpg
🚗 TOYOTA PRIUS - 104g/km - 63mpg
🚗 HYUNDAI IONIQ - 104g/km - 66mpg
🚗 TOYOTA PRIUS - 105g/km - 63mpg


,Make,Model,Vehicle Class,Engine Size(L),CO2 Emissions(g/km),Fuel Consumption Comb (mpg),Is_Hybrid
3824,HYUNDAI,IONIQ BLUE,FULL-SIZE,1.6,96,69,False
4900,HYUNDAI,IONIQ BLUE,FULL-SIZE,1.6,96,69,False
5931,HYUNDAI,IONIQ Blue,FULL-SIZE,1.6,96,69,False
6950,HYUNDAI,IONIQ Blue,FULL-SIZE,1.6,96,69,False
6949,HYUNDAI,IONIQ,FULL-SIZE,1.6,99,67,False
4899,HYUNDAI,IONIQ,FULL-SIZE,1.6,102,66,False
3823,HYUNDAI,IONIQ,FULL-SIZE,1.6,103,64,False
3229,TOYOTA,PRIUS,MID-SIZE,1.8,104,63,False
5930,HYUNDAI,IONIQ,FULL-SIZE,1.6,104,66,False
4297,TOYOTA,PRIUS,MID-SIZE,1.8,105,63,False


## 📈 결론 및 인사이트

In [9]:
print("\n✨ 🌱 주요 인사이트 🌱 ✨")
print("=" * 40)
print(f"🏆 가장 친환경 제조사: {top_15.index[0]}")
print(f"🌍 전체 평균 CO2: {co2_data['CO2 Emissions(g/km)'].mean():.1f} g/km")
print(f"♻️ 친환경 차량 비율: {len(eco_friendly)/len(co2_data)*100:.1f}%")
print(f"🔋 하이브리드 평균: {hybrid_avg:.1f} g/km")
print(f"🚗 일반차량 평균: {conventional_avg:.1f} g/km")
print(f"💚 하이브리드 개선효과: {(conventional_avg-hybrid_avg)/conventional_avg*100:.1f}%")

print("\n🎯 환경 보호 권장사항:")
print("• 하이브리드 차량 우선 선택")
print("• 2.0L 이하 소형 엔진 고려")
print("• 연비 30mpg 이상 차량 추천")
print("• CO2 200g/km 이하 목표")


✨ 🌱 주요 인사이트 🌱 ✨
🏆 가장 친환경 제조사: SMART
🌍 전체 평균 CO2: 250.6 g/km
♻️ 친환경 차량 비율: 20.2%
🔋 하이브리드 평균: 158.5 g/km
🚗 일반차량 평균: 252.3 g/km
💚 하이브리드 개선효과: 37.2%

🎯 환경 보호 권장사항:
• 하이브리드 차량 우선 선택
• 2.0L 이하 소형 엔진 고려
• 연비 30mpg 이상 차량 추천
• CO2 200g/km 이하 목표
